In [100]:
#Introduction
#A salesman wants to find a suitable house which is surrounded by nice environment, such as convenient transportation. This study aims to provide useful information for the salesman. I would like to cluster areas which have similar environmental characteristics. I hope that these information could help in determining which house is the best one to buy.

In [102]:
#Data 
#All venues of neighborhood and then categorized. Wikipedia page of Helsinki neighborhood including.

In [103]:
#Methodology
#This study provided some basic information about neighborhoods.

In [6]:
#The Battle of the Neighborhoods WEEK1
import numpy as np
import pandas as pd

In [8]:
pd.set_option('display.max_columns', None)

In [11]:
pd.set_option('display.max_rows', None)

In [12]:
import json

In [13]:
from geopy.geocoders import Nominatim

In [14]:
import requests

In [15]:
from pandas.io.json import json_normalize

In [16]:
from sklearn.cluster import KMeans

In [17]:
import folium

In [18]:
from lxml import html

In [19]:
import matplotlib.cm as cm

In [20]:
import matplotlib.colors as colors

In [21]:
import matplotlib.pyplot as plt

In [22]:
url='https://fi.wikipedia.org/wiki/Helsingin_alueellinen_jako'

In [23]:
pageContent=requests.get(url)

In [24]:
neighborhood_html=html.fromstring(pageContent.content)

In [25]:
column_names = ['Neighborhood', 'lat', 'lng'] 

In [26]:
df = pd.DataFrame(columns=column_names)

In [27]:
rows = neighborhood_html.xpath('//*[@id="mw-content-text"]/div/table[2]/tbody/tr')

In [30]:
for row in rows:
    children = row[0].getchildren()
    for child in children:
        if child.tag == 'ul':
            for li in child[0]:
                #print('Suppea')
                df = df.append({'Neighborhood': li.text}, ignore_index=True)
        else:
            df = df.append({'Neighborhood': child.text}, ignore_index=True)

In [31]:
price_url = 'https://www.asuntojenhinnat.fi/myytyjen-asuntojen-tilastot/kunta/Helsinki/'

In [32]:
pricePageContent=requests.get(price_url)

In [33]:
price_html = html.fromstring(pricePageContent.content)

In [34]:
price_column_names = ['Neighborhood', 'Price'] 

In [35]:
df_price = pd.DataFrame(columns=price_column_names)

In [38]:
hood=[]

In [39]:
price=[]

In [40]:
price_rows = price_html.xpath('//*[@id="main"]/section[4]/div/div/table/tbody/tr')

In [41]:
for row in price_rows:
    hood.append(row[1].text_content())
    price.append(row[2].text)

In [43]:
df_price['Neighborhood'] = hood

In [44]:
df_price['Price'] = price

In [45]:
df = pd.merge(df, df_price, on='Neighborhood')

In [46]:
address = 'Helsinki, FI'

In [47]:
geolocator = Nominatim(user_agent="ny_explorer")

In [48]:
location = geolocator.geocode(address)

In [49]:
latitude = location.latitude

In [50]:
longitude = location.longitude

In [51]:
lat = []

In [52]:
lng = []

In [53]:
for adr in df['Neighborhood']:
    loc = geolocator.geocode(adr)
    lat.append(loc.latitude)
    lng.append(loc.longitude)

In [54]:
df['lat'] = lat

In [55]:
df['lng'] = lng

In [56]:
CLIENT_ID = 'XDSVHHZ0OH2OITHZSB5MJSEHSUVR5J3CYY5EOHOQTV550IQ1'

In [57]:
CLIENT_SECRET = '5MNXPTBQ4GVTH0KMT0H0UKX00KYCEDLTI0XLT5BKJGME4AO3'

In [58]:
VERSION = '20180605'

In [62]:
LIMIT = 200

In [63]:
radius = 300

In [67]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'

In [68]:
results = requests.get(url).json()

In [70]:
Helsinki_map = folium.Map(location=[latitude, longitude], zoom_start=11)

In [72]:
for index, row in df.iterrows():

    adr = row['Neighborhood']
    lat = row['lat']
    lng = row['lng']

    label = '{}'.format(adr)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.9,
        parse_html=False).add_to(Helsinki_map)  

In [107]:
Helsinki_map

In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [75]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [81]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
venues = results['response']

In [90]:
nearby_venues = json_normalize(venues)

In [91]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

In [ ]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, price in zip(helsinki_merged['lat'], helsinki_merged['lng'], helsinki_merged['Neighborhood'], helsinki_merged['Cluster Labels'],helsinki_merged['Price']):
    label = folium.Popup(str(poi) + ' Most of venue type: ' + str(cluster_name[cluster]) + '. Avg square price: ' + str(price), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

In [104]:
#Resulrs
#As processed above.

In [105]:
#Discussion
#The City has some kind of venues to offer families who want to move there. However, some additional data are needed to deepen further in the future.

In [106]:
#Conclusion 
#This analysis could benefit for salesman to provide information for families who want to move here.